In [1]:
from hexathello import autoPlayer, engine, jable, printing

import numpy as np

from os import path

# -- Settings
game_size: int = 5
player_count: int = 2

In [2]:
# We want to train a KerasHexAgent. To do this, we use data we created in `quickstart_recording_data.ipynb`

history_dir: str = path.join(
    'data',
    'history',
    'examples'
)
assert path.isdir(
    history_dir
)

baseline_data_path: str = path.join(
    history_dir,
    'greendom_size-{}_players-{}.json'.format(
        game_size, player_count
    )
)
   

In [3]:
# Read the data from disk to learn from
history_fromDisk: jable.JyFrame = jable.read_file(
    baseline_data_path
)

# Decode the state, option, and play vectors from integers to numpy arrays
history_decoded: jable.JyFrame = autoPlayer.history_fromInt(
    history_fromDisk
)

assert len( history_decoded ) >= 20000
    
# Make it PoV 0 to appropriately learn
povHistory: jable.JyFrame = autoPlayer.povHistory_from_literalHistory(
    history_decoded
)

del history_decoded
del history_fromDisk

In [ ]:
# We want to train a Keras Neural Network on the data we have.
# The input size is the length of a state vector
# The output size is the length of the play vector
# Take both from the first row
input_size: int = len( povHistory[0,'board_state'] )
output_size: int = len( povHistory[0, 'player_action'] )

# The `KerasHexAgent` subclass of `HexAgent` has a `brain` property; this is the neural network
# We could in fact us any objects conforming to the `PredictionModel` protocol, which has methods:
#   - fit()
#   - predict()
#   - call()
#
# We're going to train it on the Greendom data
# Match the input to a board state vector
import tensorflow as tf
brain_input = tf.keras.layers.Input(
    shape = (input_size,),
    name = 'keras_tensor'
)

# Get creative with architecture on the inside
brain_next = tf.keras.layers.Dense(
    input_size**2,
    activation = 'relu'
)( brain_input )

brain_next = tf.keras.layers.Dense(
    input_size**2,
    activation = 'relu'
)( brain_next )

# Make the output size equal to the move vector size
brain_output = tf.keras.layers.Dense(
    output_size,
    activation = 'sigmoid'
)( brain_next )

brain_model: tf.keras.Model = tf.keras.Model(
    brain_input,
    brain_output
)
    
# Choose your learning rate and optimizer. Adam is probably good for the latter.
# You most likely want Binary Cross Entropy. Learning rate 0.0001 to 0.01 is likely fine
brain_model.compile(
    optimizer = tf.keras.optimizers.Adam(
        learning_rate = 0.005
    ),
    loss = tf.keras.losses.BinaryCrossentropy()
)

# Init the AI Agent
ai_keras: autoPlayer.KerasHexAgent = autoPlayer.KerasHexAgent(
    size = game_size,
    player_count = player_count,
    brain = brain_model,
    player_id = None,
    ai_id = 'Keras_alpha_0'
)

# Set the checkpoint to save
ai_keras_path: str = path.join(
    'data',
    'ai',
    'examples',
    '{}.keras'.format( ai_keras.ai_id )
)

ai_keras_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=ai_keras_path,
    monitor='val_binary_crossentropy',
    mode='min',
    save_best_only=True
)

ai_keras.train(
    history = povHistory,
    epochs = 10,
    callbacks = [ ai_keras_checkpoint_callback ]
)

2025-03-27 00:23:04.690695: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
103/300 ━━━━━━━━━━━━━━━━━━━━ 6:23 2s/step - loss: 0.2213

In [ ]:
# Save the network
if False:
    ai_keras.brain.save(
        path.join(
            'data',
            'ai',
            'examples',
            ai_keras.ai_id
        )
    )
#

# Get creative with the brain you use to train a KerasHexAgent, and try writing a subclass changing:
#   .prep_training_history(...)
#   .getMove_fromBoardState